In [1]:
!pip install datasets
! pip install peft
from datasets import load_dataset, load_from_disk
from peft import LoraConfig, get_peft_model
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

import torch
import os
import numpy as np

from torch.utils.data import DataLoader
from torch.nn import functional as F

import json
# Mount to google drive either click it or add a block cell

# Change it to your google drive path where this notebook located.
drive_path = '/content/drive/MyDrive/Projects/CryptoniteAnalysis/'
os.chdir(drive_path)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 6.2 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires p

## Load Model

In [2]:
"""
So far I have 4 pretrained models:
    - facebook/bart-base
    - facebook/bart-large-cnn
    - google-t5/t5-small
    - google-t5/t5-large
"""
BART_BASE = 'facebook/bart-base'
BART_LARGE_CNN = 'facebook/bart-large-cnn'
T5_SMALL = 'google-t5/t5-small'
T5_LARGE = 'google-t5/t5-large'

def get_pretrained_model_dir(model_name):
    """
    Get the path to the pretrained model.
    """
    if model_name == BART_BASE:
        return 'Baselines/Seq2Seq/TrainingData/bart-base/epoch=5_batch=16_lr=5e-05/epoch=4'
    if model_name == BART_LARGE_CNN:
        return 'Baselines/Seq2Seq/TrainingData/bart-large-cnn/epoch=3_batch=16_lr=0.0005_LoRA_teacher/epoch=2'
    if model_name == T5_SMALL:
        return 'Baselines/Seq2Seq/TrainingData/t5-small/epoch=5_batch=16_lr=5e-05_teacher/epoch=4'
    if model_name == T5_LARGE:
        return 'Baselines/Seq2Seq/TrainingData/t5-large/epoch=3_batch=16_lr=0.0005_LoRA_teacher/epoch=2'

    raise ValueError("Model name not supported")

def get_hyperparameters(model_name):
    """
    Get the config of the pretrained LoRA. So far I have 2 models thats trained with LoRA:
    - facebook/bart-large-cnn
    - google-t5/t5-large
    """
    hyperparameters_fp = None
    if model_name == BART_LARGE_CNN:
        hyperparameters_fp = 'Baselines/Seq2Seq/TrainingData/bart-large-cnn/epoch=3_batch=16_lr=0.0005_LoRA_teacher/hyper_parameters.json'
    if model_name == T5_LARGE:
        hyperparameters_fp = 'Baselines/Seq2Seq/TrainingData/t5-large/epoch=3_batch=16_lr=0.0005_LoRA_teacher/hyper_parameters.json'
    if model_name == T5_SMALL:
        hyperparameters_fp = 'Baselines/Seq2Seq/TrainingData/t5-small/epoch=5_batch=16_lr=5e-05_teacher/hyper_parameters.json'
    if model_name == BART_BASE:
        hyperparameters_fp = 'Baselines/Seq2Seq/TrainingData/bart-base/epoch=5_batch=16_lr=5e-05/hyper_parameters.json'

    if hyperparameters_fp is None:
        raise ValueError("Model name not supported")

    with open(hyperparameters_fp, 'r') as f:
        hyperparameters = json.load(f)
    return hyperparameters


def load_lora_model(model_name):
    # get the lora config for the model
    lora_config_dict = get_hyperparameters(model_name)['lora_config']
    lora_config = LoraConfig(**lora_config_dict)

    # load base model
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    base_model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

    # load the pretrained lora parameters
    pretrained_model_dir = get_pretrained_model_dir(model_name)
    lora_params_fp = os.path.join(pretrained_model_dir, "lora_params.pth")
    lora_params = torch.load(lora_params_fp, map_location=device)

    # add the lora parameters to the model
    lora_model = get_peft_model(base_model, lora_config)
    model_dict = lora_model.state_dict()
    model_dict.update(lora_params)
    lora_model.load_state_dict(model_dict)
    return lora_model, tokenizer

def load_normal_model(model_name):
    pretrained_model_dir = get_pretrained_model_dir(model_name)
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSeq2SeqLM.from_pretrained(pretrained_model_dir)
    return model, tokenizer


def load_model_and_tokenizer(model_name):
    if model_name in [BART_BASE, T5_SMALL] :
        return load_normal_model(model_name)
    if model_name in [BART_LARGE_CNN, T5_LARGE]:
        return load_lora_model(model_name)


## Load Datasets

In [3]:
def get_tokenized_dataset_fp(model_name):
    if model_name == BART_BASE:
        return 'Baselines/Seq2Seq/ProcessedDatasets/bart-base'
    if model_name == BART_LARGE_CNN:
        return 'Baselines/Seq2Seq/ProcessedDatasets/bart-large-cnn'
    if model_name == T5_SMALL:
        return 'Baselines/Seq2Seq/ProcessedDatasets/t5-small'
    if model_name == T5_LARGE:
        return 'Baselines/Seq2Seq/ProcessedDatasets/t5-large'

In [4]:
def get_dataloaders(model_name, subsample=True):
    # load the tokenized datasets
    tokenized_datasets = get_tokenized_datasets(model_name, subsample)

    # initialize dataloaders
    per_device_train_batch_size = get_hyperparameters(model_name)['per_device_train_batch_size']
    dataloaders = {}
    dataloaders['train'] = DataLoader(tokenized_datasets['train'], batch_size=per_device_train_batch_size, shuffle=True)
    dataloaders['test'] = DataLoader(tokenized_datasets['test'], batch_size=per_device_train_batch_size)
    dataloaders['validation'] = DataLoader(tokenized_datasets['validation'], batch_size=per_device_train_batch_size, shuffle=True)  # shuffle because we want to subsample

    return dataloaders


def get_tokenized_datasets(model_name, subsample=True):
    # load the preprocessed dataset
    tokenized_dataset_fp = get_tokenized_dataset_fp(model_name)
    tokenized_datasets = load_from_disk(tokenized_dataset_fp)
    tokenized_datasets.set_format("torch")
    # tokenized_datasets = tokenized_datasets.filter(lambda x: x['enumeration'] == '(9)')
    tokenized_datasets = tokenized_datasets.remove_columns(['enumeration'])

    if subsample:
        # for testing purposes
        n = 16 * 2
        tokenized_datasets['test'] = tokenized_datasets['test'].select(range(n))
        tokenized_datasets['validation'] = tokenized_datasets['validation'].select(range(n))
        tokenized_datasets['train'] = tokenized_datasets['train'].select(range(n))

    return tokenized_datasets

# Evaluation

In [5]:
def calculate_accuracy(logits, labels, tokenizer):
    '''
    There can be two way to calculate accuracy:
    1. compare what percentage of the output tokens are the same (expect for special tokens)
    If we want to compare number of tokens to be the same, then we can flatten the tokens and compare one by one.
    predictions = predictions.view(-1)
    labels = labels.view(-1)

    2. compare how many answers are correct in a batch
    Then if we want to compare answers, the dumb way is to first batch decode them, and then compare the decoded strings oe by one.
    Another way is to compare the tokens without decoding. But I am not sure how to deal with special tokens (sometimes it might not
    Generate correct end tokens.)

    '''
    predictions = torch.argmax(logits, dim=-1)

    pred_words = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    gold_standard_words = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # calculate correct predictions
    correct_labels, total_labels = 0, 0
    for i in range(len(pred_words)):
        if pred_words[i] == gold_standard_words[i]:
            correct_labels += 1
        total_labels += 1
    accuracy = correct_labels / total_labels
    return accuracy

def customize_loss_and_accuracy(outputs, target, tokenizer):
    '''
    Potential bugs
    The output of the model is by default the same length as the input sequence.
    Here by chance the input sequence is the same length as the labels (I padded them to be 40)
    If we want to be bug free for calculate accuracy, then we need to consider situations when the input sequence is not the same length to the labels.

    After we batch_decode in accuracy, the evaluation time is super high. So I suggest we don't calculate accuracy during training.
    And also I feel like we don't need to evaluate during training -- take too much time.
    '''
    # make the input and target the correct size (input is (batch* seq_len, dictionary_size), output is (batch*seq_len))
    loss = F.cross_entropy(input=outputs.logits.view(-1, outputs.logits.size(-1)), target=target.view(-1))
    accuracy = calculate_accuracy(logits=outputs.logits, labels=target, tokenizer=tokenizer)
    return loss, accuracy

def evaluate_model(model, tokenizer, dataloader):
    model.to(device)
    '''evaluate means validate or test'''
    # set model to eval mode
    model.eval()
    # calculate number of samples being evaluated
    total_validated_samples = 0
    # calculate total loss and total number of correct labels (weighted acuracy)
    total_loss = 0
    total_accurate = 0
    # turn off grad computation
    with torch.no_grad():
        # evaluate batch by batch
        for batch in dataloader:
            # put everything on the right device
            batch =  {k: v.to(device) for k, v in batch.items()}
            batch_size = batch['labels'].shape[0]

            # forward pass in the model
            outputs = model(**batch)

            # accumulate loss and accuracy
            loss, accuracy = customize_loss_and_accuracy(outputs, target=batch['labels'], tokenizer=tokenizer)
            total_loss += loss.item()
            total_accurate += accuracy * batch_size
            total_validated_samples += batch_size


    # calculate the loss and accuracy
    average_loss = total_loss/total_validated_samples
    accuracy = total_accurate/total_validated_samples

    # record the loss and accuracy
    record = {"avg_loss": average_loss, 'accuracy': accuracy}
    print(record)

    return record




# Evaluation for 4 models

In [8]:
records = {}
model_name = T5_LARGE
for model_name in [BART_BASE, BART_LARGE_CNN, T5_SMALL, T5_LARGE]:
    model, tokenizer = load_model_and_tokenizer(model_name)
    dataloaders = get_dataloaders(model_name, subsample=False)
    dataloader = dataloaders['test']
    record = evaluate_model(model, tokenizer, dataloader)
    records[model_name] = record
    del model, tokenizer, dataloaders

print(records)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.72k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

...

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.95G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

{'avg_loss': 0.014473656484027674, 'accuracy': 0.012080896127231715}
{'facebook/bart-base': {'avg_loss': 0.02670454930007223, 'accuracy': 0.013724815536949956}, 'facebook/bart-large-cnn': {'avg_loss': 0.9476232418943064, 'accuracy': 0.0}, 'google-t5/t5-small': {'avg_loss': 0.01681845185953292, 'accuracy': 0.0056581412241465}, 'google-t5/t5-large': {'avg_loss': 0.014473656484027674, 'accuracy': 0.012080896127231715}}


# Example 1
This example is used to see if I can batch decode the results. Notice: in huggingface, almost all tokenizer interit from PreTrainedTokenizer, which inherit from PreTrainedTokenizerBase.

In [10]:
model_name = BART_BASE
tokenized_datasets = get_tokenized_datasets(model_name)
model, tokenizer = load_model_and_tokenizer(model_name)
batch = tokenized_datasets['test'][2:4]
batch =  {k: v.to(device) for k, v in batch.items()}

output = model(**batch)


In [11]:
predictions = torch.argmax(output.logits, dim=-1)

print(tokenizer.batch_decode(predictions, skip_special_tokens=True))
print(tokenizer.batch_decode(batch['labels'], skip_special_tokens=True))

['traicate', 'ca mar']
['dogmatise', 'broad minded']


# Example 2
See how to get all the special tokens during iteration, so that I can skip them when calculate accuracy

In [12]:
tokenized_datasets = get_tokenized_datasets(model_name)

batch = tokenized_datasets['test'][2:4]
batch =  {k: v.to(device) for k, v in batch.items()}

output = model(**batch)
predictions = torch.argmax(output.logits, dim=-1)

tokenizer.batch_decode(batch['labels'])

['<s>dogmatise</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>',
 '<s>broad minded</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>']

In [13]:
special_tokens = tokenizer.all_special_tokens
special_token_ids = tokenizer.all_special_ids

print("Special Tokens:", special_tokens)
print("Special Token IDs:", special_token_ids)

Special Tokens: ['<s>', '</s>', '<unk>', '<pad>', '<mask>']
Special Token IDs: [0, 2, 3, 1, 50264]


## Example 3
Get total number of different enumeration values



In [14]:
# load the original dataset
def load_dataset_from_disk():
    data_dir = 'datasets/cryptonite-official-split/'
    train_fp = data_dir + 'cryptonite-train.jsonl'
    val_fp = data_dir + 'cryptonite-val.jsonl'
    test_fp = data_dir + 'cryptonite-test.jsonl'
    datasets = load_dataset('json', data_files={'train': train_fp, 'validation': val_fp, 'test': test_fp})
    return datasets

datasets = load_dataset_from_disk()


Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [19]:
unique_values = set(datasets['train']['enumeration'])
num_unique_values = len(unique_values)
num_unique_values

1588

# Example Code 1
Look at number of parameters before and after LoRA

In [16]:
def count_parameters(model):
    '''
    1. Print the model's layers
    2. Print the number of trainable/non-trainable parameters in the model.
    '''
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    non_trainable_params = sum(p.numel() for p in model.parameters() if not p.requires_grad)
    print(model)
    print(f"Trainable parameters: {trainable_params}")
    print(f"Non-trainable parameters: {non_trainable_params}")

In [17]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# define model
tokenizer = AutoTokenizer.from_pretrained("facebook/bart-large-cnn")
model = AutoModelForSeq2SeqLM.from_pretrained("facebook/bart-large-cnn")

count_parameters(model)

config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

BartForConditionalGeneration(
  (model): BartModel(
    (shared): Embedding(50264, 1024, padding_idx=1)
    (encoder): BartEncoder(
      (embed_tokens): BartScaledWordEmbedding(50264, 1024, padding_idx=1)
      (embed_positions): BartLearnedPositionalEmbedding(1026, 1024)
      (layers): ModuleList(
        (0-11): 12 x BartEncoderLayer(
          (self_attn): BartSdpaAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
          (final_l

In [18]:
rank = 32
lora_config = LoraConfig(
    r=rank,
    lora_alpha=32,  # todo: idk what this does yet
    lora_dropout=0.05, # todo: idk what this does yet
    # print(model) to see all the linear layers, and do LoRA on all of them
    target_modules=['q_proj', 'k_proj', 'v_proj', 'out_proj', 'fc1', 'fc2'],
    # unfreeze the head of the model too
    modules_to_save=['lm_head']
)
lora_model = get_peft_model(model, lora_config)
count_parameters(lora_model)

PeftModel(
  (base_model): LoraModel(
    (model): BartForConditionalGeneration(
      (model): BartModel(
        (shared): Embedding(50264, 1024, padding_idx=1)
        (encoder): BartEncoder(
          (embed_tokens): BartScaledWordEmbedding(50264, 1024, padding_idx=1)
          (embed_positions): BartLearnedPositionalEmbedding(1026, 1024)
          (layers): ModuleList(
            (0-11): 12 x BartEncoderLayer(
              (self_attn): BartSdpaAttention(
                (k_proj): lora.Linear(
                  (base_layer): Linear(in_features=1024, out_features=1024, bias=True)
                  (lora_dropout): ModuleDict(
                    (default): Dropout(p=0.05, inplace=False)
                  )
                  (lora_A): ModuleDict(
                    (default): Linear(in_features=1024, out_features=32, bias=False)
                  )
                  (lora_B): ModuleDict(
                    (default): Linear(in_features=32, out_features=1024, bias=False)
          